# NLP
When you see NLP, you should think of Text and Speech\

Text -> turn into numbers -> build a model -> train a model to find patterns -> use patterns(make prediction)\



In [8]:
# Get data (10% of labels)
import zipfile

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("nlp-getting-started.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [9]:
# turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
# test data doesn't have a target ( we want to try to predict this)
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total training samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total training samples: 10876


In [14]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # Create random index not higher than the total number
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")
    

Target: 1 (real disaster)
Text:
???? #Krefeld: the incident happened in a chemical industry park! Emergency operations underway! A building reportly collapsed! @cnnbrk @ntvde

---

Target: 1 (real disaster)
Text:
Û÷Faceless body belonged to my sisterÛª: #Hiroshima #Nagasaki #nuke survivors recall horrors 70 years on ÛÓ RT News http://t.co/918EQmTkrL

---

Target: 0 (not real disaster)
Text:
Me trying to look cute wen crush is passing by ... http://t.co/Z87zMi3Ozs

---

Target: 0 (not real disaster)
Text:
my brain id about to explode lmao

---

Target: 0 (not real disaster)
Text:
From @LeanDOTorg: Lean Thinking for Quicker Police Emergency Response Time http://t.co/suZBkyW5TT

---



### Split data into training and validation sets

In [15]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation set
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
train_df_shuffled["target"].to_numpy(),
test_size=0.1, # dedicate 10% of samples to validation set,
random_state=42)

In [16]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

In NLP there are two concepts for turning text into numbers\
<b>Tokenization</b>\
A straight mapping from word or sentence to a numerical value\
<b>Embeddings</b>\
Representation of natural language which can be learned. Representation comes in the form of a feature vector.

In [18]:
import tensorflow as tf

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary
standardize="lower_and_strip_punctuation", # how to process text
split="whitespace", # how to split tokens
ngrams=None, # create groups of n-words
output_mode="int", # how to make tokens to numbers
output_sequence_length=None) # how long should the ouput sequence of tokens be?

In [19]:
# Find average number of tokens (words) in training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [20]:
# Setup another text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be

text_vectorizer =  tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
output_mode="int",
output_sequence_length=max_length)

In [21]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [22]:
# Create sample sentence and tokenize it
sample_sentence = "do you know this is sample sentence?"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  68,   12,  106,   19,    9, 8839,    1,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

notice output_sequence_length = 15, so no matter the size of the sequence we pass to text_vectorizer, it always return a sequence with length 15

In [23]:
# choose random sentence
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
    \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@__ScrambledEggs calling it now: KIA gets banned or quarantined before month's end    

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[8739, 1705,   15,   48,    1,  742,  793,   53,  433,  239, 1119,
         304,    0,    0,    0]], dtype=int64)>

In [24]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens
bottom_5_words = words_in_vocab[-5:] # Least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer
Powerful thing about an embedding is it can be learned during training. Tokenization cannot learn(is static)\
<b>Input_dim</b> - The size of the vocabulary\
<b>Output_dim</b> - The size of the output embedding vector\
<b>Embeddings_initializer </b>- How to initialize the embedding matrix\
<b>Input_length</b> - Length of sequences being passed to embedding layer\

In [25]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # itialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")
embedding

We just created embedding tensorflow layer. We can use it as part of a model, meaning its parameters(word represent) can be updated and improved as it learns.

In [26]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
    \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
IÛªve just signed up for the Detonate Nottingham Autumn Launch Party. Register here: http://t.co/Km8uCIHrRN    

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01387816, -0.04037098,  0.02454552, ...,  0.04164531,
          0.03675271,  0.01165976],
        [-0.03624376,  0.04096097,  0.00048   , ..., -0.0216538 ,
          0.00666716,  0.01672911],
        [-0.01218056,  0.03776804, -0.042695  , ...,  0.02196519,
         -0.00387148,  0.04090047],
        ...,
        [ 0.01193949, -0.04389204,  0.00746778, ..., -0.00726794,
         -0.03415887,  0.00991349],
        [ 0.03977952, -0.03782602, -0.03646283, ...,  0.00236253,
          0.03332629,  0.02803668],
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097]]], dtype=float32)>

In [27]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.01387816, -0.04037098,  0.02454552, -0.01573527,  0.0351646 ,
        0.04676745, -0.03681671,  0.0279292 ,  0.03961677,  0.00186767,
       -0.04730154, -0.00420477, -0.04449501, -0.041125  , -0.03358554,
        0.04066365,  0.00082291,  0.02987077, -0.02802328, -0.03749008,
       -0.028752  , -0.02913274, -0.00316538,  0.04508562,  0.0367418 ,
       -0.04710525, -0.00194141, -0.0357183 ,  0.02646628, -0.04643911,
       -0.03757938, -0.01348056, -0.02931099, -0.03154454,  0.04608564,
        0.00665252, -0.03576834, -0.03499713,  0.03302273,  0.00192929,
        0.02728982,  0.01253692, -0.04105368,  0.01293231,  0.02215796,
       -0.00359243, -0.02030566, -0.04435338, -0.00800883, -0.04745622,
       -0.02592883,  0.0076615 ,  0.02814205, -0.02680153,  0.03866103,
       -0.00036297, -0.02435836, -0.0322661 ,  0.02505152, -0.04081105,
        0.01568684,  0.04301989,  0.00137084,  0.00564011, -0.00229055,
       -0.028173

This is what our computer sees each word as. When our model looks for patterns in different samples, these values will be updated as necessary.

## Modeling a text dataset

We will try out a list of models and see which one performed best\
Naive Bayes\
Feed-Forward neural network\
LSTM Model\
GRU model\
Bidirectional-LSTM Model\
1D Convolutional Neural Network\
TensorFlow Hub Pretrained Feature Extractor\


In [28]:
# Baseline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modeling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

### Creating an evaluation function for our model experiments

Let's create helper function which takes an array of predictions and ground truth labels and computes the following\
Accuracy\
Precision\
Recall\
F1-Score\

In [31]:
# Function to evaluate: accuracy, precision, recall, f1_score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classifcation model.
    
    Returns a dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {'accuracy': model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [32]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A simple dense model Neural Net

In [33]:
# Create directory to save TensorBoard Logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # Inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # Lower the dimensionality of the embedding
outputs = layers.Dense(1, activation='sigmoid')(x) # create the output layer, want binary outputs
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model


In [34]:
# Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [35]:
# Get summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Fit the model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 3s 5ms/step - loss: 0.6094 - accuracy: 0.6916 - val_loss: 0.5357 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4410 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3463 - accuracy: 0.8605 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4767 - val_accuracy: 0.7874


In [37]:
# check the results
model_1.evaluate(val_sentences, val_labels)


24/24 [==============================] - 0s 3ms/step - loss: 0.4767 - accuracy: 0.7874


[0.47668489813804626, 0.787401556968689]

In [38]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

array([[0.4048997 ],
       [0.74437505],
       [0.99789494],
       [0.1089061 ],
       [0.1114305 ],
       [0.9355648 ],
       [0.9134605 ],
       [0.99253464],
       [0.97157174],
       [0.26569813]], dtype=float32)

In [39]:
# Turn prediction probabilities into a single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

## Recurrent Neural Networks
Simply put: Use information from the past to help you with the future. Take an input and compute an output based on all previous inputs\

<b>One to one:</b> One input, one output, such as image classification\
<b>One to many:</b> One input, many output, such as image captioning\
<b>Many to one:</b> Many input, one output, such as text classification\
<b>Many to Many:</b> Many input, Many output, such as machine translation\


In [40]:
# Model 2: LSTM
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                    output_dim=128,
                                    embeddings_initializer="uniform",
                                    input_length=max_length,
                                    name='embedding_2')

# Create LSTM Model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2_LSTM')


(None, 15, 128)
(None, 64)


In [41]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [42]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 5s 8ms/step - loss: 0.5100 - accuracy: 0.7415 - val_loss: 0.4567 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3175 - accuracy: 0.8717 - val_loss: 0.5139 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2201 - accuracy: 0.9152 - val_loss: 0.5858 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1556 - accuracy: 0.9428 - val_loss: 0.6040 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1076 - accuracy: 0.9594 - val_loss: 0.8748 - val_accuracy: 0.7507


In [44]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

((762, 1),
 array([[0.00710386],
        [0.7871485 ],
        [0.99963677],
        [0.05668796],
        [0.00257999],
        [0.9996238 ],
        [0.92141753],
        [0.9997992 ],
        [0.99949515],
        [0.66567504]], dtype=float32))

In [45]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [46]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results                           

{'accuracy': 75.06561679790026,
 'precision': 0.7510077975908164,
 'recall': 0.7506561679790026,
 'f1': 0.7489268622514025}

### Model 3: GRU
GRU network has similar feature to an LSTM cell but has less parameters

In [48]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")


In [49]:
# Compile GRU model
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [50]:
# Get a summary of the GRU model
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


Notice LSTM model has more trainable parameters than the GRU cell.

In [51]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.5242 - accuracy: 0.7314 - val_loss: 0.4553 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3195 - accuracy: 0.8695 - val_loss: 0.4937 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2197 - accuracy: 0.9183 - val_loss: 0.5607 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1599 - accuracy: 0.9441 - val_loss: 0.6220 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1221 - accuracy: 0.9584 - val_loss: 0.6204 - val_accuracy: 0.7677


In [52]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

((762, 1),
 array([[0.3334687 ],
        [0.8773777 ],
        [0.9980236 ],
        [0.11562637],
        [0.01236299],
        [0.9925592 ],
        [0.62132007],
        [0.9981329 ],
        [0.9982368 ],
        [0.50158083]], dtype=float32))

In [53]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [54]:
# Calculate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.77165354330708,
 'precision': 0.7675450859410361,
 'recall': 0.7677165354330708,
 'f1': 0.7667932666650168}

## Model 4: Bidirectional RNN model
Standard RNN process a sequence from left to right, where bidirectional RNN will process the sequence from left to right and then again from right to left.


In [56]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidrectional RNN in TensorFlow
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)

x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both way so double the parameters to train
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidrectional")


In [57]:
model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [58]:
# Get a summary
model_4.summary()

Model: "model_4_Bidrectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Fit the model (takes longer because of the bidirectional layers)
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.5093 - accuracy: 0.7481 - val_loss: 0.4606 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3135 - accuracy: 0.8708 - val_loss: 0.5144 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2150 - accuracy: 0.9178 - val_loss: 0.5626 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1523 - accuracy: 0.9469 - val_loss: 0.6365 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1083 - accuracy: 0.9639 - val_loss: 0.6510 - val_accuracy: 0.7664


In [60]:
# Make predictions with bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[0.03995823],
       [0.8280176 ],
       [0.99842227],
       [0.13524732],
       [0.00311263],
       [0.9921962 ],
       [0.9553725 ],
       [0.99945563],
       [0.998982  ],
       [0.2814204 ]], dtype=float32)

In [61]:
# Convert prediction probabilities to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
# Calculate bidirectional RNN model results
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 76.64041994750657,
 'precision': 0.7665895370389821,
 'recall': 0.7664041994750657,
 'f1': 0.7651213533864446}

## Pretrained Embeddings (Transfer learning for NLP)

In [63]:
sample_sentence = ["There's a flood in my street!"]
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [64]:
# Example of pretrained embedding with universal sentence encoder - https://tfhub.dev/google/universal-sentence-encoder/4
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load Universal Sentence Encoder
embed_samples = embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

ValueError: When input_signature is provided, all inputs to the Python function must be convertible to tensors:
  inputs: (
    [["There's a flood in my street!"], 'When you call the universal sentence encoder on a sentence, it turns it into numbers.'])
  input_signature: (
    TensorSpec(shape=<unknown>, dtype=tf.string, name=None))

In [ ]:
# Each sentence has been encoded into a 512 dimension vector
embed_samples[0].shape

In [65]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") 

In [66]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_4 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [67]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 5s 16ms/step - loss: 0.5038 - accuracy: 0.7859 - val_loss: 0.4477 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4146 - accuracy: 0.8136 - val_loss: 0.4368 - val_accuracy: 0.8071
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3997 - accuracy: 0.8219 - val_loss: 0.4327 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3922 - accuracy: 0.8260 - val_loss: 0.4281 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3856 - accuracy: 0.8291 - val_loss: 0.4297 - val_accuracy: 0.8176


In [68]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.1616196 ],
       [0.75651956],
       [0.98830557],
       [0.20484091],
       [0.7304971 ],
       [0.6591135 ],
       [0.98067147],
       [0.9770989 ],
       [0.9271192 ],
       [0.08704827]], dtype=float32)

In [69]:
# Convert prediction probabilities to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [70]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 81.75853018372703,
 'precision': 0.8202616926815424,
 'recall': 0.8175853018372703,
 'f1': 0.8159820561172786}

## Making predictions on the test dataset



In [71]:
# Making predictions on the test dataset
test_sentences = test_df['text'].to_list()
test_samples = random.sample(test_sentences, 10)
for test_sample in test_samples:
    pred_prob = tf.squeeze(model_6.predict([test_sample])) # has to be a list
    pred = tf.round(pred_prob)
    print(f"Pred: {int(pred)}, Prob: {pred_prob}")
    print(f"Text:\n{test_sample}\n")
    print("----\n")


Pred: 1, Prob: 0.565223753452301
Text:
@johndavidblake Winds me up to see 95% 'whites' held up as 'black' as if just a trace of flattened nose or curly hair is impure.

----

Pred: 0, Prob: 0.20992141962051392
Text:
Pelling hotels: no strings concealment from straight a rejuvenati???ng evacuation day: pqhaxp

----

Pred: 0, Prob: 0.2170921266078949
Text:
?please read casualty fans?
Let us know &amp; help spread the word by retweeting! DM me or @ScriptetteSar for more info?? http://t.co/HDrBTJRoHb

----

Pred: 1, Prob: 0.9861007332801819
Text:
@61_alvin @Craigyb74 @nomar4eva @Shahasda80 @wherepond @trensabby @parallelpond the wall Israel erected has stopped suicide bombing?

----

Pred: 0, Prob: 0.11619587987661362
Text:
@cameron13131313 @OtooleDalton @averywelch23 @otoole1055 first of all you can't 'hit stick' shit.. Second I'm body bagging you on sight ??????

----

Pred: 0, Prob: 0.09346773475408554
Text:
@itsTiimothy bhill bruh you can obliterate beez

----

Pred: 0, Prob: 0.0715298

# Predicting on Tweets from the wild


In [73]:
# Turn Tweet into string
daniels_tweet = "Life like an ensemble: take the best choices from others and make your own"

In [74]:
def predict_on_sentence(model, sentence):
    """
    use model to make prediction on sentence.
    returns the sentence, the predicted label and the prediction probability"""
    pred_prob = model.predict([sentence])
    pred_label = tf.squeeze(tf.round(pred_prob)).numpy()
    print(f"Pred: {pred_label}", "(real_disaster)" if pred_label > 0 else "(not real disaster)", f"Prob: {pred_prob[0][0]}")
    print(f"Text:\n{sentence}")

# Make a prediction on Tweet from the wild
predict_on_sentence(model=model_6, # use the USE model
                    sentence=daniels_tweet)

Pred: 0.0 (not real disaster) Prob: 0.04917609691619873
Text:
Life like an ensemble: take the best choices from others and make your own


In [75]:
# Source - https://twitter.com/BeirutCityGuide/status/1290696551376007168
beirut_tweet_1 = "Reports that the smoke in Beirut sky contains nitric acid, which is toxic. Please share and refrain from stepping outside unless urgent. #Lebanon"

# Source - https://twitter.com/BeirutCityGuide/status/1290773498743476224
beirut_tweet_2 = "#Beirut declared a “devastated city”, two-week state of emergency officially declared. #Lebanon"

In [76]:
# Predict on diaster Tweet 1
predict_on_sentence(model=model_6, 
                    sentence=beirut_tweet_1)

Pred: 1.0 (real_disaster) Prob: 0.9664455056190491
Text:
Reports that the smoke in Beirut sky contains nitric acid, which is toxic. Please share and refrain from stepping outside unless urgent. #Lebanon


In [77]:
# Predict on diaster Tweet 2
predict_on_sentence(model=model_6, 
                    sentence=beirut_tweet_2)

Pred: 1.0 (real_disaster) Prob: 0.9755615592002869
Text:
#Beirut declared a “devastated city”, two-week state of emergency officially declared. #Lebanon
